# Qualitative comparative

Takes the predicted IMR of NEFRONA images and supepose it with the original image. In a second step, we take the predicted contourn and superpose we superpose the original image with the predicted contourn and the GT.

In [9]:
import PIL.Image as pil
import numpy as np
import os
import matplotlib.pylab as plt
import scipy as sc

In [10]:
path='/media/HDD3TB/data_carotid/auto_imt/images_enric/'
for imatge in os.listdir(path):
    #obtain the name image and their name in the GT
    name='_'.join(imatge.split('_')[-4:])
    name_gt='_'.join(name.split('_')[1:])#[:-4]+'_IMT.png'
    path_gt='/media/HDD3TB/data_carotid/GT_ACC_PL_FINAL_20180619_part1/'
    GT=os.listdir(path_gt)
    
    #Look for the GT image
    if name_gt in GT in imatge:

        im=np.asarray(pil.open(path+imatge))
        original=np.asarray(pil.open('/media/HDD3TB/data_carotid/adjusted_ENRIC/crop_original_enric/'+name))

        im_cont=np.zeros_like(im)
        #num is the half of the width of the contourn
        num=3
        im_cont[num:-num,:]=im[2*num:,:]+im[:-2*num,:]
        im_cont=im_cont-2*im

        #initialize the output image
        superpost=np.zeros((im.shape[0],im.shape[1],3),dtype='uint8')
        #we fill the image with the original image unless the contourn part which is only added in one channel (red)
        superpost[:,:,0]=original*(im_cont<255)+im_cont
        superpost[:,:,1]=original*(im_cont<255)
        superpost[:,:,2]=original*(im_cont<255)

        #read de GT and resize and crop them like the original image
        im_gt=pil.open(path_gt+name_gt)
        im_gt=im_gt.resize((884,825),resample=pil.BICUBIC)
        im_gt=np.asarray(im_gt)[94:-93,65:-64]
#         arsup=np.asarray(sup)[:,:,0:3]
        arim=255*np.asarray(pil.fromarray(im_gt))
        
        # as before we only add the GT in some channels to obtain the desidered color
        im_final=np.copy(superpost)
        im_final[arim>200]=0
        im_final[:,:,2]=im_final[:,:,2]+arim
        im_final[:,:,0]=im_final[:,:,0]+arim
        
        #save the final image
        plt.imsave('/media/HDD3TB/data_carotid/auto_imt/images_enric_superpost/'+name,im_final)
